# Handling Missing data
Real world data is hardly ever perfect. Most real-world datasets are rarely clean and homogeneous. Datasets often have missing values. Things become more complicated when different data sources indidate missing data in different ways.

We will explore some general considerations while dealing with missing data. We will see how Pandas choose to represent missing values and we will also explore built-in Pandas functionality to deal with such missing values. Missing data will be represented as *null, NaN, or NA* values.

### Some tradeoffs of using various missing data conventions
There are number of ways developed over the years to indicate presence of missing data in a table of DataFrame. Generally they deal with two strategies 

- Using a *mask* that globally indicates a missing value. In this approach, the mask might be an entirely separate boolean array or we may appropriate one bit in data represtation to locally indicate the *null* status of the value.

- Using a *sentinel* value that indicates a missing entry. In this approach, the sentinel value could be data specific convention, for example indicating a missing integer value with value `-99999` or some rare pattern, or it could be a global convention, for example a missing floating-point value can be represented with a `NaN`(Not a Number), a special value which is part of IEEE floating-point specification.

Note that, there are trade-offs with both the approaches. Using of a separate boolean array for mask requires an addition boolean array, which adds overhead in both computation and storage. On the other hand, sentinel value can restrict the range of valid values, and sometime requires intimate knowledge of the data. They may require extra often non-optimized logic in CPU and GPU arithmetic. Also note that, common special values like *NaN* might not be available for all data types.

### Missing Data in Pandas

Pandas's constraints on handling missing data comes from `NumPy` which does not have a built-in notion of *null* values. `NumPy` does have support for masked arrays, that is, arrays that have a separate Boolean mask array attached for marking data as "good" or "bad." Pandas could have derived from this, but the overhead in both storage, computation, and code maintenance makes that an unattractive choice.

Pandas chose to handle missing values through sentinel approach. It relies on two already existing *null* sentinel values in Python. The special floating point `NaN` and the python `None` object. As discussed above, this approach has limitations too but it is an overall good compromise. 


#### Pythonic missing data: ``None``

The first sentinel value used by Pandas is `None` object often used in Python to represent missing data. Because it is a Python object, None cannot be used in any arbitrary NumPy/Pandas array, but only in arrays with data type 'object' (i.e., arrays of Python objects):


In [1]:
import numpy as np
import pandas as pd

In [2]:
vals = np.array([10, None, 30, 40])
vals

array([10, None, 30, 40], dtype=object)

Notice the `dtype=object`. `NumPy` infers the contents of the array as Python object, because of a `None` type, while for a `NumPy` array without missing data, it would not be the case.

In [3]:
vals2 = np.array([10, 20, 30, 40])
vals2

array([10, 20, 30, 40])

Because of this inference, operations on such `object` type could have much more overhead then typically fast operations on the arrays with native type.

In [4]:
for dtype in ['object', 'int']:
    print("dtype:", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype: object
52.1 ms ± 573 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype: int
1.87 ms ± 23.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)



Clearly operations on `object` type are slower in magnitudes. Another unfortunate disadvantage is now we can not use aggregations like `sum()` and `mean()` across an array with a `None` value.

In [5]:
# supposed to fail
vals.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

#### Missing numerical data: ``NaN``

The other missing data representation is `NaN` (acronym for *Not a Number*). It is a special floating-point value recognized by all the systems that use IEEE floating-point representation. 

In [9]:
vals3 = np.array([10, np.nan, 30, 40])
vals3.dtype

dtype('float64')

In this case, NumPy chose native floating point instead of object type. This means that operations will be faster as compared to object type. Fair warning that `NaN` is a bit like data virus. It infects any object it touches. No matter what the operation is, arithmetic with `NaN` will be another `NaN`.

In [10]:
1 + np.nan

nan

In [11]:
0 * np.nan

nan

This also means that aggregate functions will also yield `NaN`.

In [12]:
vals3.sum(), vals3.mean(), vals3.max(), vals3.min()

C:\Users\ramand\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\numpy\core\_methods.py:28: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims, initial)
C:\Users\ramand\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\numpy\core\_methods.py:32: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims, initial)


(nan, nan, nan, nan)

However, `NumPy` does have some ways to get around this.

In [13]:
np.nansum(vals3), np.nanmean(vals3), np.nanmin(vals3), np.nanmax(vals3)

(80.0, 26.666666666666668, 10.0, 40.0)

`NaN` is a floating point value and there is no equivalent `NaN` for integers, strings, or any other types.

#### Using `NaN` and `None` in Pandas
Pandas is built to handle both `NaN` and `None` interchangeably. Both `NaN`and `None` have their place.

In [14]:
pd.Series([10, np.nan, 30, 40])

0    10.0
1     NaN
2    30.0
3    40.0
dtype: float64

When types don't have available sentinel value, Pandas automatically typecasts when there are NA values. For example if we set a value in an integer array to `NaN`, pandas will automatically upcast to a floating-point type to accomodate the NA:

In [16]:
val = pd.Series(range(3), dtype=int)
val

0    0
1    1
2    2
dtype: int32

In [18]:
val[0] = None
val

0    NaN
1    1.0
2    2.0
dtype: float64

Note that Pandas automatically converted a `None` type to `NaN` type, as it casts integer to floating point. Also note that `string` data in python is always stored as an `object` type.
As a result of this interchangeablity of `None` and `Nan` and because of upcasting, there are several methods for detecting, removing and replacing null values in pandas.

- ``isnull()``: Generate a boolean mask indicating null values.
- ``notnull()``: Opposite of `isnull()`
- ``dropna()``: Return a filtered version of data where missing values are dropped.
- ``fillna()``: Return a copy of the data with missing values filled or imputed

Let's explore these methods a little.

#### Detecting null values
Pandas data structures has two useful methods for detecting missing/null data. `isnull()` and `notnull()`. Both returns a boolean masks based on their description.

In [19]:
data = pd.Series([10, np.nan, 30, 40])
data.isnull()

0    False
1     True
2    False
3    False
dtype: bool

In [20]:
data.notnull()

0     True
1    False
2     True
3     True
dtype: bool

We can use these boolean masks directly as an index in `DataFrame` or `Series`:

In [22]:
data[data.notnull()]

0    10.0
2    30.0
3    40.0
dtype: float64

#### Dropping null values
`dropna()` (which removes NA values) and `fillna()` (which fills in NA values). For a Series, the result is straightforward:

In [23]:
data.dropna()

0    10.0
2    30.0
3    40.0
dtype: float64

For a `DataFrame` we have more options

In [25]:
df = pd.DataFrame([[10,     np.nan, 20],
                   [20,     30,     40],
                   [np.nan, 50,     60]])
df

,0,1,2
0,10.0,NaN,20
1,20.0,30.0,40
2,NaN,50.0,60


We can not drop individual `NaN` values. We can either drop row or column. `dropna()` provides options to do both on a `DataFrame`. By default, row containing missing values gets dropped.

In [26]:
df.dropna()

,0,1,2
1,20.0,30.0,40


On other hand, you can drop NA values along a different axis; `axis=1` drops all columns containing a null value:

In [27]:
df.dropna(axis='columns')

,2
0,20
1,40
2,60


But this drops some good data as well; you might rather be interested in dropping rows or columns with *all* NA values, or a majority of NA values. This can be specified through the `how` or `thresh` parameters, which allow fine control of the number of nulls to allow through.

The default is `how='any'`, such that any row or column (depending on the `axis` keyword) containing a null value will be dropped. You can also specify `how='all'`, which will only drop rows/columns that are all null values:

In [29]:
df[3] = np.nan
df

,0,1,2,3
0,10.0,NaN,20,NaN
1,20.0,30.0,40,NaN
2,NaN,50.0,60,NaN


In [31]:
df.dropna(axis='columns', how='all')

,0,1,2
0,10.0,NaN,20
1,20.0,30.0,40
2,NaN,50.0,60


For finer-grained control, the `thresh` parameter lets you specify a minimum number of non-null values for the row/column to be kept:

In [32]:
df

,0,1,2,3
0,10.0,NaN,20,NaN
1,20.0,30.0,40,NaN
2,NaN,50.0,60,NaN


We will use `thresh=3` to drop rows which have less than 3 non-null values. 

In [33]:
df.dropna(axis='rows', thresh=3)

,0,1,2,3
1,20.0,30.0,40,NaN


#### Filling null values
Sometimes, it is more useful to fill NA values with a default rather than drop them. This is where `fillna` comes in. This fill value might be a single number like zero, or it might be some sort of imputation or interpolation from the good values. We can do this in-place using the `isnull()` method as a mask, but because it is such a common operation Pandas provides the `fillna()` method, which returns a copy of the array with the `null` values replaced.

In [34]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

We can fill `NaN` with 0 in this case.

In [35]:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

We can also specify forward fill to propagate previous value forward

In [36]:
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

Similarly, backfill can be done

In [37]:
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

For `DataFrame`s, the options are similar, but we can also specify an `axis` along which the fills take place:

In [38]:
df

,0,1,2,3
0,10.0,NaN,20,NaN
1,20.0,30.0,40,NaN
2,NaN,50.0,60,NaN


In [39]:
df.fillna(method='ffill', axis=1)

,0,1,2,3
0,10.0,10.0,20.0,20.0
1,20.0,30.0,40.0,40.0
2,NaN,50.0,60.0,60.0


In [40]:
df.fillna(method='ffill', axis=0)

,0,1,2,3
0,10.0,NaN,20,NaN
1,20.0,30.0,40,NaN
2,20.0,50.0,60,NaN


Notice that NA values remain if the value is not available during forward or backward fill.